In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)

### from_template() 메소드
* 변수를 중괄호로 묶어서 템플릿에 정의

In [3]:
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
# {} 안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요?"

# from_template()을 사용해 PromptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [4]:
# prompt 생성 (완성) format() 메소드를 이용해 변수에 값을 넣음
prompt = prompt.format(name="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [8]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate.from_template(template)
print(prompt)

chain = prompt | llm

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


In [6]:
# 기본적으로는 딕셔너리 형태로 변수를 작성해야 하지만 변수가 한개일때는 변수만 작성 가능

# print(chain.invoke({"language":"Python"}).content)
print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1989년에 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성을 중요하게 생각하였고, 간결하고 명확한 문법을 지향했습니다. Python은 1991년에 첫 번째 버전이 공개되었으며, 이후로 많은 개발자들에 의해 발전해왔습니다.


### PromptTemplate 객체 생성과 동시에 prompt 생성
* input_variables 인자를 사용해 변수 지정
* 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시킴

In [9]:
# template 정의
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"]
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

**partial_variables**
* 연산 중 미리 계산된 변수를 프롬프트 템플릿에 지정해 넣을 수 있다.
* 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 사용<br>
ex) 날짜, 시간

In [14]:
# template 정의
template = "{language1}과 {language2}는 각각 누가 만들었나요?"

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2":"JAVA"  # dictionary 형태로 partial_variable 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JAVA'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [15]:
prompt.format(language1="Python")

'Python과 JAVA는 각각 누가 만들었나요?'

**partial()**
* 연산중에 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을 수 있다.

In [16]:
prompt_partial = prompt.partial(language2="JavaScript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [17]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과 JavaScript는 각각 누가 만들었나요?')

In [18]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 Python을 만들 때 코드의 가독성을 중요하게 생각하였고, 이를 통해 프로그래밍이 더 쉽고 즐거운 경험이 되기를 원했습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 넷스케이프(Netscape)에서 일하면서 웹 브라우저에서 동적인 기능을 추가하기 위해 JavaScript를 만들었습니다. 처음에는 "Mocha"라는 이름으로 시작했지만, 나중에 "LiveScript"로 변경되었고, 최종적으로 "JavaScript"라는 이름이 붙여졌습니다.


### 파일로부터 template 읽어오기
* prompt를 편하게 작성 및 수정 가능
* 상황에 맞게 파일을 작성해두면 필요할 때마다 꺼내서 쓸 수 있다.

In [20]:
from langchain_core.prompts import load_prompt

# prompt = load_prompt("경로", encoding="utf-8")
prompt = load_prompt("prompts/language_simple.yaml", encoding="utf-8")
print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에 대해 3줄로 설명해줘.'


In [21]:
prompt.format(language="Python")

'Python 언어에 대해 3줄로 설명해줘.'

In [22]:
prompt2 = load_prompt("prompts/language.yaml", encoding="utf-8")
print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요
언어의 특징을 다음의 양식에 맞게 정리하세요.
300자 내외로 작성하세요.
한글로 작성하세요.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [23]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Python")

print(answer)

1. 특징: Python은 간결하고 가독성이 높은 문법을 가진 고급 프로그래밍 언어로, 다양한 프로그래밍 패러다임(객체지향, 함수형 등)을 지원합니다. 동적 타이핑과 자동 메모리 관리를 제공하여 개발 속도를 높입니다.

2. 제작자: Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다.

3. 대표적인 프레임워크: Django, Flask, FastAPI 등 다양한 웹 프레임워크가 있으며, 데이터 과학 분야에서는 Pandas, NumPy, TensorFlow 등이 유명합니다.

4. 많이 사용되는 분야: 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립트 작성 등 다양한 분야에서 널리 사용됩니다.


### ChatPromptTemplate
* 대화 목록을 프롬프트로 주입하고자 할 때 사용 가능
* 메세지는 튜플 형태로 전달
    * ("role", "message")로 구성
    * 리스트 형태로 생성 가능

**role**
* system : 시스템 설정 메세지로 주로 전역 설정을 할 때 사용
* human: 사용자의 입력 메세지
* ai: AI의 답변 메세지

In [24]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [25]:
print(chat_prompt.format(language = "Python"))

Human: Python의 제작자는 누구인가요?


In [29]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)

messages = chat_template.format_messages(
    name="noname", user_input="당신의 이름은 무엇입니까?"
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 noname입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]


In [30]:
print(llm.invoke(messages).content)

제 이름은 noname입니다. 하지만 여러분이 원하시는 대로 불러주셔도 괜찮습니다! 어떻게 도와드릴까요?


### MessagePlaceHolder
* 아직 확정된 메세지가 아니지만, 나중에 채워질 메세지 위치를 잡아두기 위해 사용
* 보통 대화 기록을 하고 싶을 때 사용

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것 입니다."
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count}단어로 요약합니다.")
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [32]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것 입니다.
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5단어로 요약합니다.


In [33]:
chain = chat_prompt | llm | StrOutputParser()

In [34]:
input = {
    "word_count":5,
    "conversation":[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

chain.invoke(input)

'파이썬, 리스트, 중복 제거, set.'

In [36]:
!pip install langchain-community

In [38]:
from langchain_community.llms import Ollama

# model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

response

' 지구의 자전 주기는 약 24시간입니다. 이를 하루라고 부릅니다.\n\n\n자세히 알아보겠습니다:\n\n* **시차:**  지구의 자전 주기는 실제로 23시간 56분으로, 이것이 **본질적인 자전 시간**입니다. 하지만 우리가 사용하는 "하루"라는 단위는 24시간이며, 1일에서 실제 자전 시간과의 차이는 고려하여 조정되었습니다.  \n\n* **태양 기준:** 지구의 자전은 태양을 중심으로 회전하며 이를 \'자전\'이라고 합니다. 하지만 다른 행성이나 천체와 같은 다른 관점에서 본 경우, 지구의 자전 주기는 다르게 보일 수 있습니다. \n\n* **수정:**  지구의 자전 속도는 시간에 따라 변화합니다. 태양과 달의 중력이 영향을 미치며, 지각판 운동 또한 자전 속도에 영향을 미칩니다.\n\n\n 더 많은 정보가 필요하시면 언제든지 물어보세요!'

In [40]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model="gemma2:9b",
    temperature=0.1
)

In [41]:
chain = chat_prompt | model | StrOutputParser()

chain.invoke(input)

'파이썬, 중복 제거, set 사용, 간단 해결, 유용  \n'